# Predicting Car Battery Failure

Your goal in this notebook is to **predict how much time a car battery has left until it is expected to fail**. You are provided training data that includes telemetry from different vehicles, as well as the expected battery life that remains. From this you will train a model that given just the vehicle telemetry predicts the expected battery life. 

You will use compute resources provided by Azure Machine Learning (AML) to **remotely** train a **set** of models using **Automated Machine Learning**, evaluate performance of each model and pick the best performing model to deploy as a web service hosted by **Azure Container Instance**.

Because you will be using the Azure Machine Learning SDK, you will be able to provision all your required Azure resources directly from this notebook, without having to use the Azure Portal to create any resources.

## Setup
To begin, you will need to provide the following information about your Azure Subscription. 

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*). Also provide the values for the pre-created CPU cluster (`cluster_name`). 

**Be sure to replace XXXXX in the values below with your unique identifier.**

To get these values, do the following:
1. Navigate to the Azure Portal and login with the credentials provided.
2. From the left hand menu, under Favorites, select `Resource Groups`.
3. In the list, select the resource group with the name similar to `tech_immersion_XXXXX`.
4. From the Overview tab, capture the desired values.

Execute the following cell by selecting the `>|Run` button in the command bar above.

Execute the following cell by selecting the `>|Run` button in the command bar above.

In [ ]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the resource group

#Provide values for the existing Resource Group 
resource_group = "tech_immersion_XXXXX" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "tech_immersion_aml_XXXXX" # <- replace XXXXX with your unique identifier (should be lowercase)
workspace_region = "eastus2" # <- region of your resource group
#other options for region include eastus, westcentralus, southeastasia, australiaeast, westeurope

In [ ]:
# constants, you can leave these values as they are or experiment with changing them after you have completed the notebook once
experiment_name = 'automl-regression'
project_folder = './automl-regression'

# this is the URL to the CSV file containing the training data
data_url = "https://databricksdemostore.blob.core.windows.net/data/connected-car/training-formatted.csv"

# this is the URL to the CSV file containing a small set of test data
test_data_url = "https://databricksdemostore.blob.core.windows.net/data/connected-car/fleet-formatted.csv"

# provide the pre-created CPU machine learning compute found under the Compute > Training Clusters section
cluster_name = "aml-compute-cpu"
aci_service_name ='contoso-service'

### Import required packages

The Azure Machine Learning SDK provides a comprehensive set of a capabilities that you can use directly within a notebook including:
- Creating a **Workspace** that acts as the root object to organize all artifacts and resources used by Azure Machine Learning.
- Creating **Experiments** in your Workspace that capture versions of the trained model along with any desired model performance telemetry. Each time you train a model and evaluate its results, you can capture that run (model and telemetry) within an Experiment.
- Creating **Compute** resources that can be used to scale out model training, so that while your notebook may be running in a lightweight container in Azure Notebooks, your model training can actually occur on a powerful cluster that can provide large amounts of memory, CPU or GPU. 
- Using **Automated Machine Learning (AutoML)** to automatically train multiple versions of a model using a mix of different ways to prepare the data and different algorithms and hyperparameters (algorithm settings) in search of the model that performs best according to a performance metric that you specify. 
- Packaging a Docker **Image** that contains everything your trained model needs for scoring (prediction) in order to run as a web service.
- Deploying your Image to either Azure Kubernetes or Azure Container Instances, effectively hosting the **Web Service**.

In Azure Notebooks, all of the libraries needed for Azure Machine Learning are pre-installed. To use them, you just need to import them. Run the following cell to do so:

In [ ]:
!pip install --upgrade azureml-sdk==1.0.74
!pip install azureml-automl-core==1.0.74
!pip install azureml-train-automl==1.0.74  

In [ ]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import ComputeTarget
from azureml.core.webservice import Webservice
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.core import Workspace

## Create and connect to an Azure Machine Learning Workspace

Run the following cell to create a new Azure Machine Learning **Workspace** and save the configuration to disk (next to the Jupyter notebook). 

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [ ]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')


## Create a Workspace Experiment

Notice in the first line of the cell below, we can re-load the config we saved previously and then display a summary of the environment.

In [ ]:
ws = Workspace.from_config()

# Display a summary of the current environment 
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data=output, index=['']).T

Next, create a new Experiment. 

In [ ]:
experiment = Experiment(ws, experiment_name)

## Get and explore the Vehicle Telemetry Data

Run the following cell to download and examine the vehicle telemetry data. The model you will build will try to predict how many days until the battery has a freeze event. Which features (columns) do you think will be useful?

In [ ]:
data = pd.read_csv(data_url)
data

## Remotely train multiple models using Auto ML and Azure ML Compute

In the following cells, you will *not* train the model against the data you just downloaded using the resources provided by the VM. Instead, you will deploy an Azure ML Compute cluster that will download the data and use Auto ML to train multiple models, evaluate the performance and allow you to retrieve the best model that was trained. In other words, all of the training will be performed remotely with respect to this notebook. 


As you will see this is almost entirely done thru configuration, with very little code required. 

### Create the data loading script for remote compute

The Azure Machine Learning Compute cluster needs to know how to get the data to train against. You can package this logic in a script that will be executed by the compute when it starts executing the training.

Run the following cells to locally create the **get_data.py** script that will be deployed to remote compute. You will also use this script when you want train the model locally. 

Observe that the get_data method returns the features (`X`) and the labels (`Y`) in an object. This structure is expected later when you will configure Auto ML.

In [ ]:
# create project folder
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

In [ ]:
%%writefile $project_folder/get_data.py

import pandas as pd
import numpy as np

def get_data():
    
    data = pd.read_csv("https://databricksdemostore.blob.core.windows.net/data/connected-car/training-formatted.csv")
    
    X = data.iloc[:,1:73]
    Y = data.iloc[:,0].values.flatten()

    return { "X" : X, "y" : Y }

### Create AML Compute Cluster

Now you are ready to create the compute cluster. Run the following cell to create a new compute cluster (or retrieve the existing cluster if it already exists). The code below will create a *CPU based* cluster where each node in the cluster is of the size `STANDARD_DS3_V2`, and the cluster will have *1* node. 

In [ ]:
### Create AML CPU based Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

### Instantiate an Automated ML Config

Run the following cell to configure the Auto ML run. In short what you are configuring here is the training of a regressor model that will attempt to predict the value of the first feature (`Survival_in_days`) based on all the other features in the data set. The run is configured to try at most 3 iterations where no iteration can run longer that 2 minutes. 

Additionally, the data will be automatically pre-processed in different ways as a part of the automated model training (as indicated by the `preprocess` attribute having a value of `True`). This is a very powerful feature of Auto ML as it tries many best practices approaches for you, and saves you a lot of time and effort in the process.

The goal of Auto ML in this case is to find the best models that result, as measure by the normalized root mean squared error metric (as indicated by the `primary_metric` attribute). The error is basically a measure of what the model predicts versus what was provided as the "answer" in the training data. In short, AutoML will try to get the error as low as possible when trying its combination of approaches.  

The local path to the script you created to retrieve the data is supplied to the AutoMLConfig, ensuring the file is made available to the remote cluster. The actual execution of this training will occur on the compute cluster you created previously. 

In general, the AutoMLConfig is very flexible, allowing you to specify all of the following:
- Task type (classification, regression, forecasting)
- Number of algorithm iterations and maximum time per iteration
- Accuracy metric to optimize
- Algorithms to blacklist (skip)/whitelist (include)
- Number of cross-validations
- Compute targets
- Training data

Run the following cell to create the configuration.

In [ ]:
automl_config = AutoMLConfig(task = 'regression',
                             iterations = 3,
                             iteration_timeout_minutes = 5, 
                             max_cores_per_iteration = 10,
                             preprocess= True,
                             primary_metric='normalized_root_mean_squared_error',
                             n_cross_validations = 5,
                             debug_log = 'automl.log',
                             verbosity = logging.DEBUG,
                             data_script = project_folder + "/get_data.py",
                             path = project_folder)

## Run locally

You can run AutomML locally, that is it will use the resource provided by your Notebook VM environment. 

Run the following cell to run the experiment locally. Note this will take **a few minutes**.

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)
local_run

### Run our Experiment on AML Compute

Let's increase the performance by performing the training the AML Compute cluster. This will remotely train multiple models, evaluate them and allow you review the performance characteristics of each one, as well as to pick the *best model* that was trained and download it. 

We will alter the configuration slightly to perform more iterations. Run the following cell to execute the experiment on the remote compute cluster.

In [ ]:
automl_config = AutoMLConfig(task = 'regression',
                             iterations = 3,
                             iteration_timeout_minutes = 5, 
                             max_cores_per_iteration = 10,
                             preprocess= True,
                             primary_metric='normalized_root_mean_squared_error',
                             n_cross_validations = 5,
                             debug_log = 'automl.log',
                             verbosity = logging.DEBUG,
                             data_script = project_folder + "/get_data.py",
                             compute_target = compute_target,
                             path = project_folder)
remote_run = experiment.submit(automl_config, show_output=False)
remote_run

Once the above cell completes, the run is starting but will likely have a status of `Preparing` for you. To wait for the run to complete before continuing (and to view the training status updates as they happen), run the following cell. The first time you run this, it will take **about 12-15 minutes** to complete as the cluster is configured and then the AutoML job is run. Output will be streamed as the tasks progress):

In [ ]:
remote_run.wait_for_completion(show_output=True)

### List the Experiments from your Workspace

Using the Azure Machine Learning SDK, you can retrieve any of the experiments in your Workspace and drill into the details of any runs the experiment contains. Run the following cell to explore the number of runs by experiment name.

In [ ]:
ws = Workspace.from_config()
experiment_list = Experiment.list(workspace=ws)

summary_df = pd.DataFrame(index = ['No of Runs'])
pattern = re.compile('^AutoML_[^_]*$')
for experiment in experiment_list:
    all_runs = list(experiment.get_runs())
    automl_runs = []
    for run in all_runs:
        if(pattern.match(run.id)):
            automl_runs.append(run)    
    summary_df[experiment.name] = [len(automl_runs)]
    
pd.set_option('display.max_colwidth', -1)
summary_df.T

### List the Automated ML Runs for the Experiment

Similarly, you can view all of the runs that ran supporting Auto ML:

In [ ]:
proj = ws.experiments[experiment_name]
summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name'])
pattern = re.compile('^AutoML_[^_]*$')
all_runs = list(proj.get_runs(properties={'azureml.runsource': 'automl'}))
for run in all_runs:
    if(pattern.match(run.id)):
        properties = run.get_properties()
        tags = run.get_tags()
        amlsettings = eval(properties['RawAMLSettingsString'])
        if 'iterations' in tags:
            iterations = tags['iterations']
        else:
            iterations = properties['num_iterations']
        summary_df[run.id] = [amlsettings['task_type'], run.get_details()['status'], properties['primary_metric'], iterations, properties['target'], amlsettings['name']]
    
from IPython.display import HTML
projname_html = HTML("<h3>{}</h3>".format(proj.name))

from IPython.display import display
display(projname_html)
display(summary_df.T)

### Display Automated ML Run Details
For a particular run, you can display the details of how the run performed against the performance metric. The Azure Machine Learning SDK includes a built-in widget that graphically summarizes the run. 

Execute the following cell to see it.

In [ ]:
run_id = remote_run.id

from azureml.widgets import RunDetails

experiment = Experiment(ws, experiment_name)
ml_run = AutoMLRun(experiment=experiment, run_id=run_id)

RunDetails(ml_run).show() 

### Get the best run and the trained model

At this point you have multiple runs, each with a different trained models. How can you get the model that performed the best? Run the following cells to learn how.

In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

You can query for the best run when evaluated using a specific metric. 

In [ ]:
# show run and model by a specific metric
lookup_metric = "root_mean_squared_error"
best_run, fitted_model = remote_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

You can retrieve a specific iteration from a run.

In [ ]:
# show run and model from iteration 0
iteration = 0
first_run, first_model = remote_run.get_output(iteration=iteration)
print(first_run)
print(first_model)

At this point you now have a model you could use for predicting the time until battery failure. You would typically use this model in one of two ways:
- Use the model file within other notebooks to batch score predictions.
- Deploy the model file as a web service that applications can call. 

In the following, you will explore the latter option to deploy the best model as a web service.

## Download the best model 
With a run object in hand, it is trivial to download the model. 

In [ ]:
# fetch the best model
best_run.download_file("outputs/model.pkl",
                       output_file_path = "./model.pkl")

## Deploy the Model as a Web Service

Azure Machine Learning provides a Model Registry that acts like a version controlled repository for each of your trained models. To version a model, you use  the SDK as follows. Run the following cell to register the best model with Azure Machine Learning. 

In [ ]:
# register the model for deployment
model = Model.register(model_path = "model.pkl",
                       model_name = "model.pkl",
                       tags = {'area': "auto", 'type': "regression"},
                       description = "Contoso Auto model to predict battery failure",
                       workspace = ws)

print(model.name, model.description, model.version)

Once you have a model added to the registry in this way, you can deploy web services that pull their model directly from this repository when they first start up.

### Create Scoring File

Azure Machine Learning SDK gives you control over the logic of the web service, so that you can define how it retrieves the model and how the model is used for scoring. This is an important bit of flexibility. For example, you often have to prepare any input data before sending it to your model for scoring. You can define this data preparation logic (as well as the model loading approach) in the scoring file. 

Run the following cell to create a scoring file that will be included in the Docker Image that contains your deployed web service.

In [ ]:
%%writefile scoring_service.py
import pickle
import json
import numpy
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('model.pkl') # this name is model.id of model that we want to deploy
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

def run(rawdata):
    try:
        data = pd.read_json(rawdata,orient="split")
        result = model.predict(data)
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result":result.tolist()})

In [ ]:
%%writefile dockerfile
RUN apt-get update && apt-get install -y build-essential

### Deploy ACI Hosted Web Service

In [ ]:
import azureml
from azureml.core import Workspace
from azureml.core.model import Model

def getOrCreateWorkspace(subscription_id, resource_group, workspace_name, workspace_region):
    # By using the exist_ok param, if the workspace already exists we get a reference to the existing workspace instead of an error
    ws = Workspace.create(
        name = workspace_name,
        subscription_id = subscription_id,
        resource_group = resource_group, 
        location = workspace_region,
        exist_ok = True)
    return ws

def deployModelAsWebService(ws, model_folder_path="models", model_name="", 
                scoring_script_filename="scoring_service.py", 
                conda_packages=['numpy','pandas'],
                pip_packages=['azureml-sdk','onnxruntime'],
                conda_file="dependencies.yml", runtime="python",
                cpu_cores=1, memory_gb=1, tags={'name':'scoring'},
                description='Scoring web service.',
                service_name = "scoringservice"
               ):
    # notice for the model_path, we supply the name of the outputs folder without a trailing slash
    # this will ensure both the model and the customestimators get uploaded.
    print("Registering and uploading model...")
    registered_model = Model.register(model_path=model_folder_path, 
                                      model_name=model_name, 
                                      workspace=ws)

    # create a Conda dependencies environment file
    print("Creating conda dependencies file locally...")
    from azureml.core.conda_dependencies import CondaDependencies 
    mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
    with open(conda_file,"w") as f:
        f.write(mycondaenv.serialize_to_string())
        
    # create container image configuration
    print("Creating container image configuration...")
    from azureml.core.image import ContainerImage
    image_config = ContainerImage.image_configuration(execution_script = scoring_script_filename,
                                                      runtime = runtime,
                                                      conda_file = conda_file,
                                                      docker_file = 'dockerfile')
    
    # create ACI configuration
    print("Creating ACI configuration...")
    from azureml.core.webservice import AciWebservice, Webservice
    aci_config = AciWebservice.deploy_configuration(
        cpu_cores = cpu_cores, 
        memory_gb = memory_gb, 
        tags = tags, 
        description = description)

    # deploy the webservice to ACI
    print("Deploying webservice to ACI...")
    webservice = Webservice.deploy_from_model(
      workspace=ws, 
      name=service_name, 
      deployment_config=aci_config,
      models = [registered_model], 
      image_config=image_config
    )
    webservice.wait_for_deployment(show_output=True)
    
    return webservice

In [ ]:
ws =  getOrCreateWorkspace(subscription_id, resource_group, 
                   workspace_name, workspace_region)


webservice = deployModelAsWebService(ws, model_folder_path="./model.pkl", model_name="model.pkl", service_name = "predict-battery-life", 
                                     conda_packages=['numpy', 'pandas', 'scikit-learn'], pip_packages=['azureml-sdk[automl]'])

### Test the deployed web service

With the deployed web service ready, you are now ready to test calling the service with some car telemetry to see the scored results. There are three ways to approach this:
1. You could use the `Webservice` object that you acquired in the previous cell to call the service directly.
2. You could use the `Webservice` class to get a reference to a deployed web service by name.
3. You could use any client capable of making a REST call.

In this notebook, we will take the first approach. Run the following cells to retrieve the web service by name and then to invoke it using some sample car telemetry.

The output of this cell will be an array of numbers, where each number represents the expected battery lifetime in days for the corresponding row of vehicle data.

In [ ]:
%%time
# load some test vehicle data that the model has not seen
test_data = pd.read_csv(test_data_url)

# prepare the data and select five vehicles
test_data = test_data.drop(columns=["Car_ID", "Battery_Age"])
test_data.rename(columns={'Twelve_hourly_temperature_forecast_for_next_31_days_reversed': 'Twelve_hourly_temperature_history_for_last_31_days_before_death_last_recording_first'}, inplace=True)
test_data_json = test_data.iloc[:5, 0:72].to_json(orient="split")
prediction = webservice.run(input_data = test_data_json)
print(prediction)